1. Clone from [github](https://github.com/Paperspace/DataAugmentationForObjectDetection), This job is to augment image from the datasets

In [ ]:
import sys
from data_aug.data_aug import RandomHorizontalFlip, RandomHSV, RandomScale, RandomShear, Sequence

## Get Sticker

### Data augmentation flow

1. Input an Image

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt


In [ ]:
# common params
IMAGE_SIZE = 448
BATCH_SIZE = 8
NUM_CLASSES = 20
MAX_OBJECTS_PER_IMAGE = 20

# dataset params
DATA_PATH = './dataset/pascal_voc_training_data.txt'
IMAGE_DIR = './dataset/VOCdevkit_train/VOC2007/JPEGImages/'

AUG_DATASET = True
DATA_AUG_PATH = './dataset/pascal_voc_training_data_aug.txt'
IMAGE_AUG_DIR = './dataset/VOCdevkit_train/VOC2007/JPEGImages_Aug/'

# model params
CELL_SIZE = 7
BOXES_PER_CELL = 2
OBJECT_SCALE = 1
NOOBJECT_SCALE = 0.5
CLASS_SCALE = 1
COORD_SCALE = 5

# training params
LEARNING_RATE = 1e-5
EPOCHS = 40

In [ ]:
classes_name =  ["aeroplane", "bicycle", "bird", "boat", "bottle", 
                 "bus", "car", "cat", "chair", "cow", "diningtable", 
                 "dog", "horse", "motorbike", "person", "pottedplant", 
                 "sheep", "sofa", "train","tvmonitor"]

In [ ]:
from utils import *
CUT_BBOXS = False

In [ ]:
if CUT_BBOXS:
    class_counts, image_name_dict = cut_bboxs(DATA_PATH, IMAGE_DIR)
else:
    class_counts, image_name_dict = count_bboxs(DATA_PATH, IMAGE_DIR)
print(image_name_dict)



In [ ]:
import random

def choose_sticker():
    inverted_class_counts = {key: 1 / value for key, value in class_counts.items()}
    total_count = sum(inverted_class_counts.values())
    probabilities = {key: value / total_count for key, value in inverted_class_counts.items()}
    # sorted_probabilities = {k: v for k, v in sorted(probabilities.items(), key=lambda item: item[1])}
    # print(sorted_probabilities)

    # 通過值的大小當作機率選擇 key
    chosen_sticker = random.choices(list(probabilities.keys()), weights=list(probabilities.values()))[0]
    return chosen_sticker

# test = []
# for i in range(100):
chosen_sticker = choose_sticker()
    # test.append(chosen_sticker)
print("choosen sticker:", chosen_sticker)

In [ ]:
def random_image_from_folder(folder_path):
    # 檢查資料夾是否存在
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' not exist")
        return None

    # 獲取資料夾中的所有文件
    all_files = os.listdir(folder_path)

    # 過濾出圖像文件（這裡假設你的照片文件是圖像格式，如JPEG）
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    # 檢查是否有可用的圖像文件
    if not image_files:
        print(f"資料夾 '{folder_path}' 中沒有圖像文件。")
        return None

    # 從圖像文件中隨機挑選一張
    random_image = random.choice(image_files)

    # 返回完整的文件路徑
    return os.path.join(folder_path, random_image)

chosen_sticker = choose_sticker()
random_folder_path = './bboxs/origial_stickers/' + chosen_sticker
print(random_folder_path)
img = cv.imread(random_image_from_folder(random_folder_path))
plt.imshow(img)
print(img.shape)

In [ ]:
image_name_list, bboxs_list = zip(*image_name_dict.items())
image_name_list = list(image_name_list)
bboxs_list = list(bboxs_list)

print(image_name_list)
print(bboxs_list)

In [ ]:
object_num_list = [len(bboxs) for bboxs in bboxs_list]
print(object_num_list)

object_class_list = [[bbox[-1] for bbox in bboxs] for bboxs in bboxs_list]
print(object_class_list)

In [ ]:
aug_image_name_list = []
class_count = np.zeros(NUM_CLASSES, int)
total_image_num = len(image_name_list)
image_count = np.zeros(total_image_num, int)

for i in range(total_image_num):
    class_list = object_class_list[i]
    aug_image_name_list.append(f'{image_count[i]}_{image_name_list[i]}')
    image_count[i] += 1
    for c in class_list:
        class_count[c] += 1

LOWER_BOUND = 3000
sorted_indices = np.argsort(object_num_list)[::-1]
invalid_aug = np.zeros(total_image_num, int)
while True:
    for i in sorted_indices:
        class_list = object_class_list[i]
        if invalid_aug[i] or (class_count[class_list] >= LOWER_BOUND).any():
            invalid_aug[i] = 1
            continue
        else:
            aug_image_name_list.append(f'{image_count[i]}_{image_name_list[i]}')
            image_count[i] += 1
            for c in class_list:
                class_count[c] += 1
    if (class_count >= LOWER_BOUND).all():
        break

for r in range(5):
    for c in range(NUM_CLASSES // 5):
        i = r + c * 5
        print(f'{i:2d}) {classes_name[i]:11} {class_count[i]:4d}', end=('\n' if c == 3 else '\t'))

In [ ]:
import os
import cv2 as cv
import tensorflow as tf
import numpy as np

# BACKGROUND_IMAGE_PATH = './dataset/VOCdevkit_train/VOC2007/JPEGImages/'
IMAGE_AUG_PATH = './dataset/VOCdevkit_train/VOC2007/JPEGImages_Aug_with_All/' # destination to save image
DATA_AUG_PATH = './dataset/pascal_voc_training_data_aug_with_All.txt'

def image_aug():

    seq = Sequence([RandomHSV(50,50,50),
                    RandomHorizontalFlip(0.5),
                    RandomScale(0.2),
                    RandomShear(0.2)])
    
    
    # seq = Sequence([RandomHSV(50,50,50),
                    # RandomHorizontalFlip(0.5),
                    # RandomScale(0.2, 2.0)])

    with open (DATA_AUG_PATH,'w') as fout:
        
        for name in aug_image_name_list:
            num_count, origin_name = name.split('_')
            input_image_name = IMAGE_DIR + origin_name
            input_tf_image = tf.io.read_file(input_image_name)
            input_image = tf.io.decode_jpeg(input_tf_image, channels=3)
            input_bboxes = np.array(image_name_dict.get(origin_name)).astype(float)



            if num_count != '0': # need to do augmentation
                aug_image, new_bboxes = seq(input_image.numpy().copy(), input_bboxes.copy())

                # TODO: paste sticker into background image

                aug_image = tf.io.encode_jpeg(aug_image)
                tf.io.write_file(os.path.join(IMAGE_AUG_PATH, name), aug_image)
                buff_list = [name] + [str(int(b)) for box in new_bboxes for b in box]
                fout.write(' '.join(buff_list) + '\n')

            else: # no need to augmentation
                origin_img = tf.io.encode_jpeg(input_image)
                tf.io.write_file(os.path.join(IMAGE_AUG_PATH, name), origin_img)
                buff_list = [name] + [str(int(b)) for box in input_bboxes for b in box]
                fout.write(' '.join(buff_list) + '\n')
            

image_aug()